In [1]:
import keras
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd

In [3]:
df_google = pd.read_csv('../data/mined/GOOGLE1.csv')

In [4]:
import tensorflow as tf
import tensorflow_hub as hub

In [5]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/2'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/2'.


In [222]:
embeddings = elmo(
["the cat is on the mat", "dogs are in the fog"],
signature="default",
as_dict=True)["default"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
sess = tf.Session()

In [8]:
sess.run(tf.global_variables_initializer())

In [224]:
sess.run(embeddings).shape

(2, 1024)

In [10]:
tokens_input = [["the", "cat", "is", "on", "the", "mat"],
["dogs", "are", "in", "the", "fog", ""]]
tokens_length = [6, 5]
embeddings = elmo(
inputs={
"tokens": tokens_input,
"sequence_len": tokens_length
},
signature="tokens",
as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [13]:
output = sess.run(embeddings)

In [15]:
np.shape(output)

(2, 6, 1024)

In [17]:
output[0].shape

(6, 1024)

In [97]:
dfText = pd.read_csv('./mined/Combined/GoogleProcessed.csv')
dfScore = pd.read_csv('./processedSelected/googl.csv')

In [98]:
dfText['EndDate'] = pd.to_datetime(dfText['EndDate'], format='%m%d%Y')
dfText['StartDate'] = pd.to_datetime(dfText['StartDate'], format='%m%d%Y')
dfScore['Date'] = pd.to_datetime(dfScore['Date'])
dfFinal = pd.merge(dfText, dfScore[['Date','FractionalIncrease','Positive']],how='left', left_on ='EndDate', right_on='Date').drop('Date',axis=1)
dfFinal = dfFinal.dropna()

In [215]:
x = dfFinal['Text'].values
x = [x_.replace('\r\n','').replace('\xa0','')for x_ in x]
y = dfFinal['FractionalIncrease'].values
yBinary = dfFinal['Positive'].values
yBinary = [int(y)for y in yTrainBinary]

In [93]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras
import numpy as np

In [216]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [217]:
X_train, X_test, y_train, y_test = (np.asarray(X_train).reshape(-1,1), 
                                    np.asarray(X_test).reshape(-1,1), 
                                    np.asarray(y_train).reshape(-1,1), 
                                    np.asarray(y_test).reshape(-1,1))

In [174]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

In [218]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)
def ELMoEmbedding(x):
    squeezed = tf.squeeze(tf.cast(x, tf.string))
    return embed(squeezed, signature="default", as_dict=True)["default"]

In [221]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
# pred = Dense(2, activation='softmax')(dense)
pred = Dense(1, activation='linear')(dense)
model = Model(inputs=input_text, outputs=pred)
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

SyntaxError: invalid syntax (<ipython-input-221-07d1695deaf0>, line 1)

In [225]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(X_train, y_train, epochs=1, batch_size=2)
    model.save_weights('./elmo-model.h5')

Epoch 1/1
  66/1085 [>.............................] - ETA: 36:02 - loss: 0.4250 - mean_squared_error: 0.4250

KeyboardInterrupt: 

In [ ]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

data = pd.read_csv('spam.csv', encoding='latin-1')

y = list(data['v1'])
x = list(data['v2'])

le = preprocessing.LabelEncoder()
le.fit(y)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le, ['ham', 'spam', 'ham', 'ham'])

untest = decode(le, test)

x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[:5000])
y_train = np.asarray(y_enc[:5000])

x_test = np.asarray(x_enc[5000:])
y_test = np.asarray(y_enc[5000:])

from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=1, batch_size=32)
    model.save_weights('./elmo-model.h5')

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo-model.h5')  
    predicts = model.predict(x_test, batch_size=32)

y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

In [38]:
dfScore['FractionalIncrease'].value

,Date,Open,Delta,FractionalIncrease,High,Low,Positive
0,2016-11-17,782.50,-27.50,-0.033951,810.060,743.590,False
1,2016-11-25,782.61,0.11,0.000141,793.770,772.650,True
2,2016-12-01,778.55,-4.06,-0.005188,799.740,773.145,False
3,2016-12-08,792.95,14.40,0.018496,792.000,753.360,True
4,2016-12-15,817.36,24.41,0.030784,824.300,787.905,True
5,2016-12-22,809.10,-8.26,-0.010106,823.000,804.500,False
6,2016-12-29,802.33,-6.77,-0.008367,816.000,802.440,False
7,2017-01-05,807.50,5.17,0.006444,813.430,789.620,True
8,2017-01-12,828.38,20.88,0.025858,830.430,805.920,True
9,2017-01-19,829.00,0.62,0.000748,834.650,821.010,True
